In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import json
import os.path
import logging
logging.basicConfig(level=logging.WARNING) # WARNING, INFO, DEBUG

## Load Data

In [ ]:
input_file = 'input_1.csv'
if os.path.isfile(input_file):
    predictions = pd.read_csv(input_file) # Check if file exists in current directory
else:
    predictions = pd.read_csv('https://fh-public.s3-eu-west-1.amazonaws.com/ml-engineer/' + input_file)


In [ ]:
predictions.head()

In [ ]:
constraints = requests.get('https://fh-public.s3-eu-west-1.amazonaws.com/ml-engineer/constraints.json').json()
constraints

## Define function to compute 

In [ ]:
from scipy.special import lambertw

def get_accuracy_for(rate, interval):
    """
        Computes the average accuracy for a price given expected 
        rate of change and interval between updates
    """
    return (1 - np.exp(-rate * interval)) / (rate * interval)


def get_ui_for(rate, accuracy):
    """
        function to get update interval given expected rate and accuracy target
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.lambertw.html
    
    """
    W = np.real(lambertw(-np.exp(-1 / accuracy) / accuracy, k=0))
    return ((accuracy * W) + 1) / (accuracy * rate)

## Define utitily functions

In [ ]:
# Here and below Criteria_1.sum() = "average-accuracies" - "target_accuracy"
# Criteria_2.sum() = total_rpm
def update_data(_data, _target_accuracy):
    _data['rpm'] = 1 / _data['update_interval']
    _data['e_clicks_per_rpm'] = _data['e_clicks'] / _data['rpm']
    _data['accuracy'] = get_accuracy_for (_data['e_change_rate'], _data['update_interval'] / 60)
    _data['crit1']    = _data['e_clicks'] * (_data['accuracy'] - _target_accuracy)
    _data['crit2']    = 1 / _data['update_interval']
    
def log_iteration_results(_clicks, _crit1, _crit2, _len_selected):
    logging.info("clicks = %.2f, crit_1 = %.2f, crit_2 = %.2f, len(selected) = %d" % (_clicks, _crit1, _crit2, _len_selected))

## Define functions to probe new ui values

In [ ]:
def find_better_ui_one_step(_data, _lr_minus, _lr_plus, _constraints):
    """
        Basic idea is to devide itineraries in two parts:
        "update_interval" for the 1st part is decreased (with corresponding accuracy increased)
        and for the 2nd part is increased (with corresponding accuracy decreased)
    """
    local_data = pd.DataFrame()
    step_minus = _lr_minus * _data['update_interval'] 
    step_plus  = _lr_plus  * _data['update_interval'] 
    
    local_data['ui_minus'] = _data['update_interval'] - step_minus
    local_data['ui_plus']  = _data['update_interval'] + step_plus
    acc_minus = get_accuracy_for(_data['e_change_rate'], local_data['ui_minus'] / 60)
    acc_plus  = get_accuracy_for(_data['e_change_rate'], local_data['ui_plus']  / 60)
    
    local_data['cr_1_step_minus']    = _data['e_clicks'] * (acc_minus - _data['accuracy'])
    local_data['cr_1_step_plus']     = _data['e_clicks'] * (acc_plus  - _data['accuracy'])
    local_data['cum_cr1_step_minus'] = local_data['cr_1_step_minus'].cumsum()
    
    total_sum = local_data['cr_1_step_plus'].sum()
    local_data['cum_cr1_step_plus']  = total_sum + local_data['cr_1_step_plus'] - local_data['cr_1_step_plus'].cumsum()

    # Find splitting number N:
    # the first one, which gives crit_1 > 0,
    # hence giving the largest room for crit_2 reduction
    sum_col = local_data['cum_cr1_step_minus'].values[:len(local_data)-1] + local_data['cum_cr1_step_plus'].values[1:]
    N = np.arange(len(local_data) - 1)[sum_col > 0][0] + 1
    logging.debug("chosen N = %d" % N)
    
    # Update ui and calculate crit_2, which might be more than desired theshold
    ui_minus_col = local_data.columns.get_loc('ui_minus')
    ui_plus_col  = local_data.columns.get_loc('ui_plus')
    new_ui       = local_data.iloc[:N, ui_minus_col].append(local_data.iloc[N:, ui_plus_col])
    
    crit2_val = (1/new_ui).sum()       
    to_return = {'is_succeed' : True, 'new_ui' : new_ui, 'crit2' : crit2_val} 
    if (crit2_val > _constraints['max_rpm']):
        to_return['is_succeed'] = False
        
    logging.debug("crit1 = %.2f, crit2 = %.2f" % (sum_col[N], crit2_val)) 
    return to_return

def find_better_ui(_selected, _constraints, _lr = 1e-3):
    """
        Sort selected dataframe to further split itineraries into two parts:
        1. ones where accuracy should be increased (with idx <= some_threshold_N)
        2. ones where accuracy should be decreased (with idx > some_threshold_N)
    """
    _selected.sort_values('e_clicks')
    
    # Try increasing/decreasing accuracies for itineraries.
    # "minus"/"plus" in result's columns stands for reducing/increasing update_intervals
    iters     = 0
    max_iters = 15
    while (True):
        logging.debug("trying lr_minus = %f, lr_plus = %f" % (_lr, _lr))
        try_result = find_better_ui_one_step(_selected, _lr, _lr, _constraints)
        if try_result['is_succeed'] or iters > max_iters:
            break
        _lr   /= 2
        iters += 1
    if try_result['is_succeed']:
        _selected['update_interval'] = try_result['new_ui']
        update_data(_selected, _constraints['target_accuracy'])
    return try_result['is_succeed']


## Define optimization high-level routines

In [ ]:
def select_best_itinerary(_data, _max_rpm):
    bl_clicks_per_sorted = _data.sort_values('e_clicks_per_rpm', ascending=False)
    bl_clicks_per_sorted['cum_requests'] = bl_clicks_per_sorted['rpm'].cumsum()
    selected = bl_clicks_per_sorted[bl_clicks_per_sorted['cum_requests'] < _max_rpm]
    return selected

def optimization_iteration(_bl, _selected, _constraints, _learn_rate = 0.9):
    """
        Given selected itineraries set try updating it's ui
        with saving crit_1 almost the same, but decreasing crit_2.
        If it is impossible then 'is_succeed' is False
    """
    is_succeed = find_better_ui(_selected, _constraints, _learn_rate)
    
    if is_succeed:   
        # Once one has a room for crit_2
        # select additional itineraries and update baseline set
        selected_new = select_best_itinerary(_bl, _constraints['max_rpm'] - _selected['crit2'].sum())
        _bl.loc[selected_new.index, 'update_interval'] = selected_new['update_interval']
        update_data(_bl, _constraints['target_accuracy'])
        _bl.drop(selected_new.index, inplace = True)
        logging.debug("old len selected = %d, len(sel_1) = %d" %(len(selected), len(selected_new)))
        
        # Append new selected set to old one
        _selected = _selected.append(selected_new)
        log_iteration_results(_selected['e_clicks'].sum(), _selected['crit1'].sum(), _selected['crit2'].sum(), len(_selected))
        
    return _selected

## Baseline solution

In [ ]:
bl = predictions.copy()

"""
    Iteration 0
    Create baseline set of itineraries
    and select ones that satisfy criterias (crit1 and crit2)
"""

bl['update_interval'] = 60 * get_ui_for(bl['e_change_rate'], constraints['target_accuracy'])
update_data(bl, constraints['target_accuracy'])
selected = select_best_itinerary(bl, constraints['max_rpm'])
bl = bl.drop(selected.index)
log_iteration_results(selected['e_clicks'].sum(), selected['crit1'].sum(), selected['crit2'].sum(), len(selected))
logging.warning("\nBaseline clicks num = %.2f\n" % selected['e_clicks'].sum())

## Optimized solution

In [ ]:
# Loop until no new itineraries are selected 
old_size  = len(selected)
curr_size = old_size + 1
while curr_size > old_size:
    # Iteration 1, 2, ...
    old_size  = len(selected)
    selected  = optimization_iteration(bl, selected, constraints)
    curr_size = len(selected)
logging.warning("\nOptimized clicks num = %.2f" % selected['e_clicks'].sum())